In [22]:
import requests
import json
import pandas as pd

In [23]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)

In [24]:
# fetch api key from file

with open('secret/key.txt', 'r') as file:
    api_key = file.read()
#api_key

In [25]:
#keywords = ["swollen ankles", "horny goat weed", "scalp psoriasis"]

url = "https://google.serper.dev/search"
location = "Calgary,Alberta,Canada"
language = "en"
device = "desktop"

In [26]:
# peforms a serch on one kw and returns the Xth page of relults

def search_kw(kw, page=1, url="https://google.serper.dev/search"):
    payload = json.dumps({
      "q": kw, # keyword to search for
      "gl": "us", # location
      "hl": "en", # language
      "autocorrect": False,
      "page": page
    })

    headers = {
      "X-API-KEY": api_key,
      "Content-Type": "application/json"
    }

    response = requests.request("POST", url=url, headers=headers, data=payload)
    return(response)

In [27]:
# runs search_ky over multiple pages and returns them in a list

def get_pages(page_count, kw, url="https://google.serper.dev/search"):
    pages = list()
    for page in range(1, page_count+1):
        pages.append(search_kw(kw, page, url))
    return pages

In [28]:
# extract kw, page, rank, and link from 1 page of response info and return df

def extract_response(response):
    df = pd.DataFrame(columns = ['kw', 'page', 'rank','link'])
    q = response.json()['searchParameters']['q']
    page = response.json()['searchParameters']['page']
    for rank, result in enumerate(response.json()["organic"], 1):
        link = result['link']
        df.loc[len(df)] = [q, page, rank, link]
    return(df)

In [29]:
# takes a list of pages of response, iterates extract_response(), extracts and renumbers responses, returns df of ky, rank, link

def collate_pages(pages, max_rank=None):
    df = pd.DataFrame()
    for response in pages:
        data = extract_response(response)
        df = pd.concat([df, data], ignore_index=True)
    df = df.reset_index(drop=True)
    df["rank"] = df.index + 1
    if max_rank is not None:
        df = df[df['rank'] <= max_rank]
    df.drop(["page"], axis=1, inplace=True)
    return df

In [30]:
# accepty a df containing "Keywords" and iterates collate_pages() returns df of kw

def get_links_for_kws(df_in):
    df_out = pd.DataFrame()
    keywords = df_in["Keyword"]
    for kw in keywords:
        kw_pages = get_pages(3, kw) # get 3 pages (<= 30 records) for each kw
        df = collate_pages(kw_pages, 20) # colate pages, limit to top 20 records.
        df_out = pd.concat([df_out, df], ignore_index=True)
    return(df_out)

In [31]:
# load kw
df_in = pd.read_csv('data/kw_1_000.csv')
#df_in = df_in.head() # FOR TESTING PURPOSES ONLY BECAUSE I AM LIMITED BY MY QUERY QUOTA OF 1440
#df_in.iloc[400:500]

In [32]:
df_400 = df_in.iloc[0:400]
df_400_800 = df_in.iloc[400:800]
df_800_1000 = df_in.iloc[800:]

In [33]:
#df_400_800.tail()

In [ ]:
# REMEMBER YOU ARE LIMITED BY A DAILY QUERY QUOTA OF 1440

In [20]:
#df = get_links_for_kws(df_400)
df = get_links_for_kws(df_400_800)
#df = get_links_for_kws(df_800_1000)

In [21]:
#df.to_csv('data/first_400.csv')
df.to_csv('data/kw_400_800.csv')

In [36]:
df

,kw,rank,link
0,basement finishing,1,https://www.thisoldhouse.com/basements/2101784...
1,basement finishing,2,https://www.hgtv.com/design/rooms/other-rooms/...
2,basement finishing,3,https://youtube.com/watch?v=x5NWrVvi1_M
3,basement finishing,4,https://www.familyhandyman.com/project/how-to-...
4,basement finishing,5,https://www.bobvila.com/articles/how-much-does...
...,...,...,...
7995,casodex,16,https://www.medicines.org.uk/emc/product/5491/...
7996,casodex,17,https://youtube.com/watch?v=eC7ILqWgyGw
7997,casodex,18,https://en.wikipedia.org/wiki/Bicalutamide
7998,casodex,19,https://www.empr.com/drug/casodex/


In [35]:
# prints the response for 1 keyword

def print_response(response):
    print (f"Results for keyword: \"{response.json()['searchParameters']['q']}\"")
    for rank, result in enumerate(response.json()["organic"], 1):
        print(f"Rank: {rank}, Title: {result['title']}, Link: {result['link']}")